### Data Wrangling Webscraped Data:

Objective is to format, clean, and fill webscraped data.

* Remove Noise from all columns in the dataframe; i.e. unnecessary brackets, parentheses, ect


* Spot ambiguity between values and format them respectively


* Fill empty values; NaN values are filled with average of the respective column

In [1]:
# import required modules
import numpy as np
import pandas as pd

path = 'spacex_web_scrapedV3.csv'
df = pd.read_csv(path)

In [2]:
df.head()

,Date and time ( ),Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing
0,"4 June 2010,18:45","CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,F9 v1.0[7]B0003.1[8],Failure[9][10](parachute)
1,"8 December 2010,15:43[13]","CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),NaN,LEO (ISS),NASA (COTS)\nNRO,Success[9],F9 v1.0[7]B0004.1[8],Failure[9][14](parachute)
2,"22 May 2012,07:44[17]","CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],F9 v1.0[7]B0005.1[8],No attempt
3,"8 October 2012,00:35[21]","CCAFS,SLC-40",SpaceX CRS-1[22](Dragon C103),"4,700 kg (10,400 lb)",LEO (ISS),NASA (CRS),Success,F9 v1.0[7]B0006.1[8],No attempt
4,"1 March 2013,15:10","CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg (10,752 lb)",LEO (ISS),NASA (CRS),Success,F9 v1.0[7]B0007.1[8],No attempt


In [3]:
df.dtypes

Date and time ( )    object
Launch site          object
Payload              object
Payload mass         object
Orbit                object
Customer             object
Launch outcome       object
Version Booster      object
Booster landing      object
dtype: object

### Data Wrangling:

In [4]:
# fomatting variable names to PamelCase, as well as removing spaces
df.columns = df.columns.str.strip().str.title().str.replace(' ', '_', regex = True)
df.columns = df.columns.str.strip().str.replace('(\_)', '', regex = True)
df.columns = df.columns.str.strip().str.replace('(\(\))', '', regex = True)
# dropping variables: Payload, Customer, 
df.drop(columns = ['Payload', 'Customer', 'LaunchOutcome'], inplace = True)
df.rename(columns = {'BoosterLanding':  'Outcome', 
                     'PayloadMass': 'PayloadMass_kg'}, inplace = True)

df.head()

,DateAndTime,LaunchSite,PayloadMass_kg,Orbit,VersionBooster,Outcome
0,"4 June 2010,18:45","CCAFS,SLC-40",NaN,LEO,F9 v1.0[7]B0003.1[8],Failure[9][10](parachute)
1,"8 December 2010,15:43[13]","CCAFS,SLC-40",NaN,LEO (ISS),F9 v1.0[7]B0004.1[8],Failure[9][14](parachute)
2,"22 May 2012,07:44[17]","CCAFS,SLC-40","525 kg (1,157 lb)[19]",LEO (ISS),F9 v1.0[7]B0005.1[8],No attempt
3,"8 October 2012,00:35[21]","CCAFS,SLC-40","4,700 kg (10,400 lb)",LEO (ISS),F9 v1.0[7]B0006.1[8],No attempt
4,"1 March 2013,15:10","CCAFS,SLC-40","4,877 kg (10,752 lb)",LEO (ISS),F9 v1.0[7]B0007.1[8],No attempt


In [5]:
'''cleaning rows & formatting data that contain noise; i.e. brackets, 
        unstructured paradigms, ect'''
# DateAndTime column containing brackets
for n in ['(\[\d\d])', '(\[\d\d\d])', '(\(planned\))']:
    df.iloc[:, 0] = df.iloc[:, 0].str.replace(n,'', regex = True)

df.iloc[:, 0] = df.iloc[:, 0].str.replace('2020','2020,', regex = True)
df.iloc[:, 0] = df.iloc[:, 0].str.replace('2021','2021,', regex = True)
df.iloc[:, 0] = df.iloc[:, 0].str.replace(',,',',', regex = True)

# splitting time and date into separate columns
df[['Date', 'Time']] = df.iloc[:, 0].str.split(',', expand = True)
df.drop(columns = ['DateAndTime'], inplace = True)

# LaunchSite columns containing brackets
df.iloc[:, 0] = df.iloc[:, 0].str.replace('(\[\d\d\d])','', regex = True)

# cleaning and formatting Orbit column
df.iloc[:, 2] = df.iloc[:, 2].str.replace('(\[\d\d\d])','', regex = True)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('LEO (ISS)', 'ISS', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('Polar orbit LEO', 'PO', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('Polar LEO', 'PO', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('Heliocentric0.99–1.67 AU(close to Mars transfer orbit)', 'HEO', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('HEO for P/2 orbit', 'HEO', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('HEO(Sun–Earth L1 insertion)','ES-L1', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('LEO / MEO','VLEO', regex = False)
df.iloc[:, 2] = df.iloc[:, 2].str.replace('Sub-orbital', 'SO', regex = False)

In [6]:
'''removing noise from Outcome column; turning values to categorical variables 
0 for Failure Landing, 1 for Successful Landing'''

for n in ['(\[\d\d\d])', '(\[\d\d])', '(\[\d])', '(\[d\])', 
          '(\[i\])', '(\(drone ship\))', '(\(ground pad\))', 
          '(\(parachute\))', '(\(ocean\))']:
    df.iloc[:, 4] = df.iloc[:, 4].str.strip().str.replace(n,'', regex = True)

for n in ['Precluded', 'Uncontrolled', 
          'Failure', 'No attempt']:
    df.iloc[:, 4] = df.iloc[:, 4].str.strip().str.replace(n, '0', regex = True)

for n in ['Success', 'Controlled']:
    df.iloc[:, 4] = df.iloc[:, 4].str.strip().str.replace(n, '1', regex = True)
df.head(10)

,LaunchSite,PayloadMass_kg,Orbit,VersionBooster,Outcome,Date,Time
0,"CCAFS,SLC-40",NaN,LEO,F9 v1.0[7]B0003.1[8],0,4 June 2010,18:45
1,"CCAFS,SLC-40",NaN,ISS,F9 v1.0[7]B0004.1[8],0,8 December 2010,15:43
2,"CCAFS,SLC-40","525 kg (1,157 lb)[19]",ISS,F9 v1.0[7]B0005.1[8],0,22 May 2012,07:44
3,"CCAFS,SLC-40","4,700 kg (10,400 lb)",ISS,F9 v1.0[7]B0006.1[8],0,8 October 2012,00:35
4,"CCAFS,SLC-40","4,877 kg (10,752 lb)",ISS,F9 v1.0[7]B0007.1[8],0,1 March 2013,15:10
5,"VAFB,SLC-4E","500 kg (1,100 lb)",PO,F9 v1.1[7]B1003[8],0,29 September 2013,16:00
6,"CCAFS,SLC-40","3,170 kg (6,990 lb)",GTO,F9 v1.1B1004,0,3 December 2013,22:41
7,"CCAFS,SLC-40","3,325 kg (7,330 lb)",GTO,F9 v1.1,0,6 January 2014,22:06
8,"Cape Canaveral,LC-40","2,296 kg (5,062 lb)[45]",ISS,F9 v1.1,1,18 April 2014,19:25
9,"Cape Canaveral,LC-40","1,316 kg (2,901 lb)",LEO,F9 v1.1,1,14 July 2014,15:15


In [7]:
# stripping all values after 2 in column VersionBooster;
df.iloc[:, 3] = df.iloc[:, 3].str[:2]

In [8]:
# based off Wiki, Falcon Heavy uses three Falcon 9 (F9) boosters; for simplicity we will name Fa ---> Falcon 9
for n in ['Fa', 'F9']:
    df.iloc[:, 3] = df.iloc[:, 3].str.replace(n, 'Falcon 9', regex = True)

df.iloc[:, 3].unique()

array(['Falcon 9'], dtype=object)

In [9]:
df.iloc[:, 1] = df.iloc[:, 1].str[:10]
for n in ['(\(\))', 'kg', ',', '\xa0', 'xa0 79', 
          '~', '–600', '(', ' 1', ' 79']:
    df.iloc[:, 1] = df.iloc[:, 1].str.strip().str.replace(n, '', regex = True)

df.iloc[:, 1] = df.iloc[:, 1].str.strip().str.replace('Classified', '0', regex = True)

df.iloc[:, 1] = df.iloc[:, 1].astype('float')

print("Before: ", df.isna().sum())
mean = df.iloc[:, 1].mean()

df.iloc[:, 1].fillna(mean, inplace = True)
print("\nAfter: ", df.isna().sum())

Before:  LaunchSite        0
PayloadMass_kg    2
Orbit             0
VersionBooster    0
Outcome           0
Date              0
Time              0
dtype: int64

After:  LaunchSite        0
PayloadMass_kg    0
Orbit             0
VersionBooster    0
Outcome           0
Date              0
Time              0
dtype: int64


In [10]:
print("Before: \n", df.dtypes)

df.Outcome = df.Outcome.astype('int')
df.Date = pd.to_datetime(df['Date'])
df.drop(columns = 'Time', inplace = True)

print("\nAfter: \n", df.dtypes)

Before: 
 LaunchSite         object
PayloadMass_kg    float64
Orbit              object
VersionBooster     object
Outcome            object
Date               object
Time               object
dtype: object

After: 
 LaunchSite                object
PayloadMass_kg           float64
Orbit                     object
VersionBooster            object
Outcome                    int64
Date              datetime64[ns]
dtype: object


In [11]:
df.tail(20)

,LaunchSite,PayloadMass_kg,Orbit,VersionBooster,Outcome,Date
105,"CCSFS,SLC-40",7000.0,GTO,Falcon 9,1,2020-12-13
106,"KSC,LC-39A",0.0,LEO,Falcon 9,1,2020-12-19
107,"CCSFS,SLC-40",3500.0,GTO,Falcon 9,1,2021-01-08
108,"KSC,LC-39A",15600.0,LEO,Falcon 9,1,2021-01-20
109,"CCSFS,SLC-40",5000.0,SSO,Falcon 9,1,2021-01-24
110,"CCSFS,SLC-40",15600.0,LEO,Falcon 9,1,2021-02-04
111,"CCSFS,SLC-40",15600.0,LEO,Falcon 9,0,2021-02-16
112,"KSC,LC-39A",15600.0,LEO,Falcon 9,1,2021-03-04
113,"CCSFS,SLC-40",15600.0,LEO,Falcon 9,1,2021-03-11
114,"KSC,LC-39A",15600.0,LEO,Falcon 9,1,2021-03-14


In [18]:
df['FlightNo'] = list(range(1, 126))
df.head()

,LaunchSite,PayloadMass_kg,Orbit,VersionBooster,Outcome,Date,FlightNo
0,"CCAFS, SLC-40",7109.837398,LEO,Falcon 9,0,2010-06-04,1
1,"CCAFS, SLC-40",7109.837398,ISS,Falcon 9,0,2010-12-08,2
2,"CCAFS, SLC-40",525.000000,ISS,Falcon 9,0,2012-05-22,3
3,"CCAFS, SLC-40",4700.000000,ISS,Falcon 9,0,2012-10-08,4
4,"CCAFS, SLC-40",4877.000000,ISS,Falcon 9,0,2013-03-01,5


In [13]:
# four launch facilities: 
# ---- Cape Canaveral Space Launch Complex 40 (SLC-40)
# ---- Vandenberg Space Force Base Space Launch Complex 4E (SLC-4E)
# ---- Kennedy Space Center Launch Complex 39A (LC-39A)
for n in ['CCSFS,SLC-40', 'Cape Canaveral,SLC-40', 'CCAFSSLC-40', 'CCAFS,SLC-40', 'Cape Canaveral,LC-40']:
    df.iloc[:, 0] = df.iloc[:, 0].str.strip().str.replace(n, 'CCAFS, SLC-40', regex = True)
df.iloc[:, 0].unique()

array(['CCAFS, SLC-40', 'VAFB,SLC-4E', 'KSC,LC-39A'], dtype=object)

In [14]:
# number of launches on each site
df.groupby(['FlightNo'])['LaunchSite'].unique().value_counts(normalize=True)

[CCAFS, SLC-40]    0.584
[KSC,LC-39A]       0.288
[VAFB,SLC-4E]      0.128
Name: LaunchSite, dtype: float64

In [15]:
# number and occurrences of each orbit
df.groupby(['FlightNo'])['Orbit'].unique().value_counts()

[LEO]      39
[GTO]      35
[ISS]      28
[PO]        8
[SSO]       7
[MEO]       3
[HEO]       2
[ES-L1]     1
[VLEO]      1
[SO]        1
Name: Orbit, dtype: int64

In [16]:
# calculate the number and occurrences of mission outcome per orbit type
outcome_perOrbit = pd.DataFrame(df.groupby(['Orbit'])['Outcome'].value_counts(normalize = True))
outcome_perOrbit['OrbitCount'] = pd.DataFrame(df.groupby(['Orbit'])['Outcome'].value_counts())
outcome_perOrbit

Outcome  OrbitCount
Orbit Outcome                      
ES-L1 1        1.000000           1
GTO   1        0.571429          20
      0        0.428571          15
HEO   0        0.500000           1
      1        0.500000           1
ISS   1        0.607143          17
      0        0.392857          11
LEO   1        0.871795          34
      0        0.128205           5
MEO   1        0.666667           2
      0        0.333333           1
PO    1        0.625000           5
      0        0.375000           3
SO    0        1.000000           1
SSO   1        0.857143           6
      0        0.142857           1
VLEO  0        1.000000           1

In [ ]:
df.to_csv("spacex_data_wrangled_V5.csv", index=False)